### Sistema de recomendción y filtrado colaborativo. Recomendación de articulos de moda

In [ ]:
conda install pyspark

In [4]:
conda install pyspark[sql]

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [4]:
conda install -c conda-forge findspark

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\garcjo30\Documents\Python

  added / updated specs:
    - findspark


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.12.0               |   py39hcbf5309_0         1.0 MB  conda-forge
    findspark-2.0.1            |     pyhd8ed1ab_0           8 KB  conda-forge
    python_abi-3.9             |           2_cp39           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.0 MB

The following NEW packages will be INSTALLED:

  findspark          conda-forge/noarch::findspark-2.0.1-pyhd8ed1ab_0
  python_abi         conda-forge/win-64::python_abi-3.9-2_cp39

The following packages will be SUPERSEDED by a higher-priority channel:

  conda              pkgs/main::conda-4.12.0-py39haa95532_0 --> conda-for

In [1]:
import pyspark
from pyspark.ml.recommendation import ALS
#from pyspark.mllib.recommendation import ALS
from pyspark.sql import SparkSession, Window
import os
import pandas as pd
from pyspark.sql.functions import lit, row_number, monotonically_increasing_id, col,isnan, when, count, desc, sum, rand, explode
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType,BooleanType,DateType
from pyspark.ml.evaluation import RegressionEvaluator
import findspark
from pyspark.ml.fpm import FPGrowth
import random

Setup 'JAVA_HOME'

In [2]:
findspark.init()

In [3]:
os.environ['JAVA_HOME'] = 'C:/Users/garcjo30/Documents/jdk-11.0.15_windows-x64_bin/jdk-11.0.15'
os.environ['PYSPARK_SUBMIT_ARGS'] = "--driver-memory 30g --executor-memory 30g pyspark-shell"
#os.environ['PYSPARK_DRIVER_PYTHON'] = "jupyter"
#os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = "notebook"
#os.environ['PYSPARK_PYTHON'] = "python"

In [4]:
#Conextion
#spark = SparkSession.builder.getOrCreate()
spark = SparkSession.builder.config("spark.driver.memory", "30g").getOrCreate()

In [5]:
spark

Lectura de los datos

In [7]:
transactions = transactions.select('customer_id','article_id')

In [8]:
transactions = transactions.withColumn("flag", lit(1))

In [9]:
transactions.limit(10).toPandas()

,customer_id,article_id,flag
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,1
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,1
2,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,1
3,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,1
4,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,1
5,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687001,1
6,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221001,1
7,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,688873012,1
8,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,501323011,1
9,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,598859003,1


In [10]:
transactions = transactions.groupBy(["customer_id","article_id"]).agg(F.sum("flag").alias("purchase_count"))

In [10]:
transactions.show()

+--------------------+----------+--------------+
|         customer_id|article_id|purchase_count|
+--------------------+----------+--------------+
|00401a367c5ac085c...| 427114015|             1|
|00b10bf5c51e961bd...| 562245050|             1|
|02042b44255a14412...| 628930004|             1|
|04cec5f3914316cc2...| 580041001|             1|
|0653814d364bd9ce3...| 306847002|             2|
|06884807549bd7463...| 669363001|             1|
|069784a67fe0e349f...| 562245042|             1|
|08065e7733fe8cc99...| 655784001|             2|
|085b8cdf4101b203c...| 652731008|             1|
|0864c58e052b24327...| 478549001|             1|
|0942b84e4d9375e47...| 564311002|             1|
|0a552cb2fd184643f...| 685687001|             1|
|0a6823e56068de750...| 623059004|             1|
|0b026732d189dfed3...| 675359001|             1|
|0b374e930ae439810...| 576719002|             1|
|0bb90665088cc97cd...| 625939014|             1|
|0beaf6fdd7de8d6c6...| 640021005|             1|
|0c631a706869dde70..

Agregamos id a customer_id para hacer más amigable el uso del codigo de cliente. 

In [11]:
costumer_df = transactions.select('customer_id').distinct()

In [12]:
costumer_df = costumer_df.withColumn("client_id", F.monotonically_increasing_id())

Cada cliente tiene un client_id único que le represneta de menera numérica

In [25]:
costumer_df.show()

+--------------------+---------+
|         customer_id|client_id|
+--------------------+---------+
|1e93a1eb9f7d4d435...|        0|
|4fe809e46c6363eb8...|        1|
|fc97786ea8cabd8c0...|        2|
|65028198b4bf902d3...|        3|
|b073b24fd8d486474...|        4|
|39958372a3f741f40...|        5|
|55ee9ece320b8324e...|        6|
|c3e6cd3801c306da4...|        7|
|05aa3c881787b2e4f...|        8|
|8da8bc7d1942f3bfc...|        9|
|afec5c4c71f2ee23b...|       10|
|9d36c90a6e413f6bd...|       11|
|e0dc06d6508538bbd...|       12|
|321c73ab8b8b365d0...|       13|
|6c4b668b05d2f8567...|       14|
|20d8178fd4d25d012...|       15|
|e4abd7221a0ec9693...|       16|
|6ee8826f6e84c4383...|       17|
|9acf97dc20386d391...|       18|
|bd585d43e61d13ab6...|       19|
+--------------------+---------+
only showing top 20 rows



In [13]:
transactions_df = transactions.join(costumer_df,transactions.customer_id == costumer_df.customer_id,"left")

In [38]:
transactions_df.show()

+--------------------+----------+--------------+--------------------+---------+
|         customer_id|article_id|purchase_count|         customer_id|client_id|
+--------------------+----------+--------------+--------------------+---------+
|000346516dd355b40...| 827636001|             1|000346516dd355b40...|     2371|
|000346516dd355b40...| 666084001|             1|000346516dd355b40...|     2371|
|000346516dd355b40...| 557248003|             1|000346516dd355b40...|     2371|
|000346516dd355b40...| 485176004|             1|000346516dd355b40...|     2371|
|000346516dd355b40...| 766955002|             1|000346516dd355b40...|     2371|
|000346516dd355b40...| 685823003|             1|000346516dd355b40...|     2371|
|000346516dd355b40...| 816423002|             1|000346516dd355b40...|     2371|
|000346516dd355b40...| 854193002|             1|000346516dd355b40...|     2371|
|000346516dd355b40...| 694966001|             1|000346516dd355b40...|     2371|
|000346516dd355b40...| 534210011|       

In [14]:
transactions_df = transactions_df.select('client_id', 'article_id', 'purchase_count')

In [34]:
transactions_df=transactions_df.withColumn("article_id",transactions_df.article_id.cast('integer'))
transactions_df=transactions_df.withColumn("client_id",transactions_df.client_id.cast('integer'))
transactions_df=transactions_df.withColumn("purchase_count",transactions_df.purchase_count.cast('integer'))

### Modelo

In [35]:
transactions_df = spark.read.csv("Data_sets_articulos/transactions_underwear.csv", header = "true")

In [37]:
transactions_df=transactions_df.withColumn("article_id",transactions_df.article_id.cast('integer'))
transactions_df=transactions_df.withColumn("client_id",transactions_df.client_id.cast('integer'))
transactions_df=transactions_df.withColumn("purchase_count",transactions_df.purchase_count.cast('integer'))

In [38]:
transactions_df

DataFrame[client_id: int, article_id: int, purchase_count: int]

In [39]:
(training, test) = transactions_df.randomSplit([0.8, 0.2])

In [44]:
als = ALS(maxIter=10, regParam=0.05, implicitPrefs=True, alpha = 80, rank = 10, 
          userCol="client_id", itemCol="article_id", ratingCol="purchase_count",
          coldStartStrategy="nan", nonnegative = True)

In [45]:
model = als.fit(training)

In [130]:
predictions = model.transform(test)

In [56]:
predictions.show()

+---------+----------+--------------+------------+
|client_id|article_id|purchase_count|  prediction|
+---------+----------+--------------+------------+
|      148| 126589006|             1| 0.020087441|
|    81284| 126589006|             1|0.0046000816|
|    36819| 126589006|             3| 0.004901303|
|     4097| 126589006|             2| 0.023317078|
|    45999| 126589006|             1|0.0043827416|
|     4452| 126589006|             1| 0.021620117|
|    21087| 126589006|             1| 0.010084717|
|    87634| 126589006|             2|0.0070427945|
|    24575| 126589006|             1|0.0085751945|
|    55152| 126589006|             1|0.0042971233|
|     3885| 126589006|             1| 0.021440638|
|    36024| 126589006|             1| 0.004695392|
|    16075| 126589006|             1| 0.016745925|
|    24072| 126589006|             1| 0.012340558|
|     6572| 126589006|             1|    0.019411|
|    55451| 126589006|             1| 0.004430715|
|     9919| 126589006|         

In [106]:
predictions.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in predictions.columns]).show()

+---------+----------+--------------+----------+
|client_id|article_id|purchase_count|prediction|
+---------+----------+--------------+----------+
|        0|         0|             0|     40179|
+---------+----------+--------------+----------+



In [48]:
print((predictions.count(), len(predictions.columns)))

(445175, 4)


El mix entre el set de entrenamiento y el de prueba es bueno, ya que únicamente se tienen 1313 valores sin el cálculo de una predicción. Para fines practicos en estos casos se colocara el valor de 1 por default. 

In [89]:
predictions_test = predictions.where(isnan("prediction"))

In [60]:
print((predictions_test.count(), len(predictions_test.columns)))

(40179, 4)


In [91]:
predictions_test.show()

+---------+----------+--------------+----------+--------------------+-----------+
|client_id|article_id|purchase_count|prediction|                rand|prediction2|
+---------+----------+--------------+----------+--------------------+-----------+
|   802713| 309434011|             1|       NaN| 0.26162677745559826|       rand|
|  1038180| 464297034|             1|       NaN|0.016160906016991716|       rand|
|   574990| 464297034|             1|       NaN|  0.3242352705869308|       rand|
|  1034305| 464297034|             1|       NaN| 0.14745346916270952|       rand|
|  1046728| 464297034|             2|       NaN| 0.17121444098114957|       rand|
|   970578| 464297034|             1|       NaN|  0.5500414038954883|       rand|
|   118683| 464297034|             1|       NaN|  0.8206651957480466|       rand|
|  1041245| 464297034|             1|       NaN|   0.158731562146729|       rand|
|  1060695| 464297034|             1|       NaN|  0.7971083662645247|       rand|
|  1037803| 4642

In [95]:
def na_val(a):
    if a.isnull():
        return random.uniform(0,1)

In [82]:
predictions = predictions.withColumn("rand", rand())

In [83]:
predictions.show()

+---------+----------+--------------+-----------+--------------------+
|client_id|article_id|purchase_count| prediction|                rand|
+---------+----------+--------------+-----------+--------------------+
|   549115| 309434011|             1|        0.0| 0.17167690229317434|
|   205929| 309434011|             1|9.481082E-5|  0.9548637055417158|
|   802713| 309434011|             1|        NaN| 0.26162677745559826|
|   417183| 464297034|             1|        0.0|  0.7604739884732543|
|   605135| 464297034|             7|        0.0| 0.14175877674124504|
|  1054640| 464297034|             1|        0.0|  0.6557620293025815|
|   728517| 464297034|             1|        0.0|  0.6347657894164226|
|   405281| 464297034|             2|        0.0|  0.6260815008192218|
|   141714| 464297034|             1|        0.0|  0.8935744416007677|
|   163792| 464297034|             1|        0.0|  0.7610023809529602|
|  1040997| 464297034|             1|        0.0|  0.8031399399126765|
|   24

In [97]:
predictions=predictions.withColumn('prediction2', F.when(isnan('prediction'), col('rand')).otherwise(col("prediction")))

In [98]:
predictions.show()

+---------+----------+--------------+-----------+--------------------+--------------------+
|client_id|article_id|purchase_count| prediction|                rand|         prediction2|
+---------+----------+--------------+-----------+--------------------+--------------------+
|   549115| 309434011|             1|        0.0| 0.17167690229317434|                 0.0|
|   205929| 309434011|             1|9.481082E-5|  0.9548637055417158|9.481082088313997E-5|
|   802713| 309434011|             1|        NaN| 0.26162677745559826| 0.26162677745559826|
|   417183| 464297034|             1|        0.0|  0.7604739884732543|                 0.0|
|   605135| 464297034|             7|        0.0| 0.14175877674124504|                 0.0|
|  1054640| 464297034|             1|        0.0|  0.6557620293025815|                 0.0|
|   728517| 464297034|             1|        0.0|  0.6347657894164226|                 0.0|
|   405281| 464297034|             2|        0.0|  0.6260815008192218|          

In [99]:
predictions = predictions.select('client_id', 'article_id', 'prediction2')

In [100]:
predictions = predictions.withColumnRenamed('prediction2', 'prediction')

In [101]:
predictions

DataFrame[client_id: int, article_id: int, prediction: double]

In [102]:
predictions.show()

+---------+----------+--------------------+
|client_id|article_id|          prediction|
+---------+----------+--------------------+
|   549115| 309434011|                 0.0|
|   205929| 309434011|9.481082088313997E-5|
|   802713| 309434011| 0.26162677745559826|
|   417183| 464297034|                 0.0|
|   605135| 464297034|                 0.0|
|  1054640| 464297034|                 0.0|
|   728517| 464297034|                 0.0|
|   405281| 464297034|                 0.0|
|   141714| 464297034|                 0.0|
|   163792| 464297034|                 0.0|
|  1040997| 464297034|                 0.0|
|   245466| 464297034|                 0.0|
|    51754| 464297034|                 0.0|
|   895718| 464297034|                 0.0|
|  1038180| 464297034|0.016160906016991716|
|   498682| 464297034|                 0.0|
|   539389| 464297034|                 0.0|
|   162514| 464297034|                 0.0|
|   574990| 464297034|  0.3242352705869308|
|  1034305| 464297034| 0.1474534

In [35]:
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.2337670324870431


Contrucción de metrica de validación Rank Ordering Error Metric

In [25]:
#w = Window.partitionBy(predictions.client_id).orderBy(prediction.value) #assuming default ascending order
w = Window.partitionBy('client_id').orderBy(desc('prediction'))

In [26]:
predictions = predictions.withColumn("perc_by_clt_id", F.percent_rank().over(w))

In [27]:
predictions.show()

+---------+----------+--------------+----------+--------------------+
|client_id|article_id|purchase_count|prediction|      perc_by_clt_id|
+---------+----------+--------------+----------+--------------------+
|      148| 673396002|             2| 1.3635318|                 0.0|
|      148| 484398001|             1| 1.3448731|3.653635367190354...|
|      148| 692930001|             1| 1.3347859|7.307270734380709E-4|
|      148| 673677002|             1| 1.3339019|0.001096090610157...|
|      148| 610776001|             2| 1.3334247|0.001461454146876...|
|      148| 610776001|             1| 1.3334247|0.001461454146876...|
|      148| 615141002|             1| 1.3313997|0.002192181220314...|
|      148| 749699002|             1|  1.326562|0.002557544757033...|
|      148| 759871002|             1| 1.3263025|0.002922908293752...|
|      148| 759871002|             1| 1.3263025|0.002922908293752...|
|      148| 759871002|             1| 1.3263025|0.002922908293752...|
|      148| 72012500

In [28]:
predictions = predictions.withColumn("r*rank", predictions.purchase_count*predictions.perc_by_clt_id)

In [29]:
predictions

DataFrame[client_id: int, article_id: int, purchase_count: int, prediction: float, perc_by_clt_id: double, r*rank: double]

Calculo ROEM

In [30]:
numerador = predictions.agg({'r*rank': 'sum'}).collect()[0][0]

In [32]:
denominador = predictions.agg({'purchase_count': 'sum'}).collect()[0][0]

In [33]:
print("ROEM: ", numerador * 1.0/denominador )

ROEM:  0.49166709968738637


Como podemos observar el valor del ROEM esta ligeramente por debajo de 0.5, lo que significa que nuestro sistema de recomendación es casi igual a dar recomendaciones al azar. La cuestión entonces es saber si podemos mejorar la recomendación. 

### Buscando el mejor modelo

In [108]:
def prediction_nan(predictions):
    predictions = predictions.withColumn("rand", rand())
    predictions = predictions.withColumn('prediction2', F.when(isnan('prediction'), col('rand')).otherwise(col("prediction")))
    predictions = predictions.select('client_id', 'article_id', 'prediction2')
    predictions = predictions.withColumnRenamed('prediction2', 'prediction')
    return predictions

In [15]:
#Calculo del ROEM
def ROEM(predictions):
    w = Window.partitionBy('client_id').orderBy(desc('prediction'))
    predictions = predictions.withColumn("perc_by_clt_id", F.percent_rank().over(w))
    predictions = predictions.withColumn("r*rank", predictions.purchase_count*predictions.perc_by_clt_id)
    numerador = predictions.agg({'r*rank': 'sum'}).collect()[0][0]
    denominador = predictions.agg({'purchase_count': 'sum'}).collect()[0][0]
    return numerador * 1.0/denominador  

In [131]:
predictions = prediction_nan(predictions)

In [132]:
predictions

DataFrame[client_id: int, article_id: int, purchase_count: int, prediction: double]

In [110]:
predictions.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in predictions.columns]).show()

+---------+----------+----------+
|client_id|article_id|prediction|
+---------+----------+----------+
|        0|         0|         0|
+---------+----------+----------+



In [46]:
ROEM(predictions)

0.49221447500380505

Definimos los distintos valores a probar

In [7]:
ranks = [10, 20, 30, 40]
maxIters = [10, 20]
regParams = [.05, .1, .15]
alphas = [20, 40, 60, 80]

In [13]:
# For loop will automatically create and store ALS models
model_list = []
for r in ranks:
    for mi in maxIters:
        for rp in regParams:
            for a in alphas:
                model_list.append(ALS(userCol= "client_id", itemCol= "article_id", ratingCol= "purchase_count", rank = r, maxIter = mi, regParam = rp, alpha = a, coldStartStrategy="drop", nonnegative = True, implicitPrefs = True))

In [9]:
# Print the model list, and the length of model_list
print (model_list, "Length of model_list: ", len(model_list))


[ALS_6f53cdabda97, ALS_ae48e424e103, ALS_1c1a60177a7f, ALS_a003d81589ea, ALS_d4a1c898dc7d, ALS_81a2b492ae2f, ALS_b2d1c08d925d, ALS_095733ddf5e8, ALS_f2cb3a247fa8, ALS_ac07b9d10f09, ALS_392667f08698, ALS_e2366b84f867, ALS_2934c2b37804, ALS_466a6633b4a5, ALS_fe47413616ea, ALS_d901483fa357, ALS_1aa45d2e990c, ALS_56157fe34093, ALS_8b5aa47f94f2, ALS_99288e0297f8, ALS_8e66ec5bea10, ALS_dd981df9e27e, ALS_733d2ca1963e, ALS_4ca7ba1df88c, ALS_5ec2107b6e3a, ALS_67631726b628, ALS_33b94a247429, ALS_3888df15f5f4, ALS_1b239f96312d, ALS_fa676f84396d, ALS_be20abc30660, ALS_d31e726d7d24, ALS_1819669205bf, ALS_926ed3c4763b, ALS_1f3cad81b6c3, ALS_c41e2120d9de, ALS_9dd38f8e5c2d, ALS_829f5fa30b0d, ALS_0d5031ac01e9, ALS_1b611752452e, ALS_9370f18a41bc, ALS_776255a36dde, ALS_c4f0484cffb9, ALS_55484541f267, ALS_e704ae5fce74, ALS_78c2ddfea683, ALS_d72be1bbf556, ALS_3204b1ffbbd8, ALS_3def7abdbce1, ALS_36f97bba76a8, ALS_1b792027f745, ALS_f6fb4e81369d, ALS_a5811b10d727, ALS_7613ddee6b25, ALS_41f216c604cd, ALS_825c0

In [10]:
# Validate
len(model_list) == (len(ranks)*len(maxIters)*len(regParams)*len(alphas))

True

In [ ]:
model_list_test = model_list[18:97]
model_list_test

In [23]:
model_list_comp[0]

ALS_ffb764d1a580

In [24]:
model_list_comp = model_list[12:193]
model_list_comp

[ALS_df6ea789b627,
 ALS_9d94df131428,
 ALS_05258afe7897,
 ALS_a779797fa72b,
 ALS_752e115ce5ee,
 ALS_599a11f4b70a,
 ALS_7736994d8f87,
 ALS_8af5a169c89d,
 ALS_2946ab8a220e,
 ALS_86f0ce52940c,
 ALS_fca0d978f2ed,
 ALS_a5bebecdf302,
 ALS_f380e021c285,
 ALS_0dd6ffdfd403,
 ALS_c38d289d579d,
 ALS_cde010b6b82f,
 ALS_183548ce9fb6,
 ALS_7a32246222c6,
 ALS_e1dd3675813b,
 ALS_f10f9f1a847c,
 ALS_b57552d6e310,
 ALS_b000dc84dcbc,
 ALS_3018300254b8,
 ALS_a1eb71050c9c,
 ALS_98977fb647ce,
 ALS_220c595c7162,
 ALS_5646cedbdfe2,
 ALS_4e7456551fff,
 ALS_7a6a611b5c73,
 ALS_8e4af4adf7f1,
 ALS_275e8b459704,
 ALS_a20c66b60814,
 ALS_b2c8771b40da,
 ALS_d4893499d8b2,
 ALS_39a614b0b352,
 ALS_d5510554f69f,
 ALS_c8e8f4d68069,
 ALS_d06d077b13c9,
 ALS_f9899ac21fda,
 ALS_7d8309d8011d,
 ALS_5aac4e3c1bd5,
 ALS_a9926f21e243,
 ALS_a7f5f3a07786,
 ALS_3939aaca1267,
 ALS_2179e8757eff,
 ALS_994832ebba80,
 ALS_87055211cbd1,
 ALS_cc1c9f74c5d6,
 ALS_fb8549d19928,
 ALS_7b498b4b3877,
 ALS_4d907ab83394,
 ALS_8b80cd443a72,
 ALS_5e06e9f

In [51]:
# Lista donde guardar los resultados
ROEMS = []

In [ ]:
# Buscamos a través de los distintos modelos
for als in model_list:
    #Ajuste del modelo
    fitted_model = als.fit(training)
    predictions = fitted_model.transform(test)
    ROEM_v = ROEM(predictions)

    # Guardamos los resultados y los vamos mostrando
    ROEMS.append(ROEM_v)
    print ("Validation ROEM: ", ROEM_v)

In [38]:
als = ALS(userCol= "client_id", itemCol= "article_id", ratingCol= "purchase_count", 
          rank = 10, maxIter = 20, regParam = 0.1, alpha = 40, 
          coldStartStrategy="drop", nonnegative = True, implicitPrefs = True)

In [39]:
model = als.fit(training)

In [40]:
predictions = model.transform(test)

In [ ]:
ROEM(predictions)

In [41]:
w = Window.partitionBy('client_id').orderBy(desc('prediction'))   

In [42]:
predictions = predictions.withColumn("perc_by_clt_id", F.percent_rank().over(w))

In [43]:
predictions = predictions.withColumn("r*rank", predictions.purchase_count*predictions.perc_by_clt_id)

In [44]:
predictions.agg({'r*rank': 'sum'}).show()

+-----------------+
|      sum(r*rank)|
+-----------------+
|2939481.161661525|
+-----------------+



In [46]:
predictions.agg({'purchase_count': 'sum'}).show()

+-------------------+
|sum(purchase_count)|
+-------------------+
|            6297629|
+-------------------+



### Funciones antes de iterar

In [6]:
def prediction_nan(predictions):
    predictions = predictions.withColumn("rand", rand())
    predictions = predictions.withColumn('prediction2', F.when(isnan('prediction'), col('rand')).otherwise(col("prediction")))
    predictions = predictions.select('client_id', 'article_id','purchase_count','prediction2')
    predictions = predictions.withColumnRenamed('prediction2', 'prediction')
    return predictions

In [7]:
#Calculo del ROEM
def ROEM(predictions):
    w = Window.partitionBy('client_id').orderBy(desc('prediction'))
    predictions = predictions.withColumn("perc_by_clt_id", F.percent_rank().over(w))
    predictions = predictions.withColumn("r*rank", predictions.purchase_count*predictions.perc_by_clt_id)
    numerador = predictions.agg({'r*rank': 'sum'}).collect()[0][0]
    denominador = predictions.agg({'purchase_count': 'sum'}).collect()[0][0]
    return numerador * 1.0/denominador  

**Modelo por tipo de producto. Zapatos**

In [135]:
ranks = [10, 20, 30, 40]
maxIters = [5, 10]
regParams = [.001, .01, .1]
alphas = [1, 10, 100]

In [136]:
# For loop will automatically create and store ALS models
model_list = []
for r in ranks:
    for mi in maxIters:
        for rp in regParams:
            for a in alphas:
                model_list.append(ALS(userCol= "client_id", itemCol= "article_id", ratingCol= "purchase_count", rank = r, maxIter = mi, regParam = rp, alpha = a, coldStartStrategy="drop", nonnegative = True, implicitPrefs = True))

In [137]:
print (model_list, "Length of model_list: ", len(model_list))

[ALS_824e163fcd90, ALS_37ef56342b97, ALS_d4db8ba00480, ALS_43bacbc47777, ALS_6071fb8fe13e, ALS_f416c23dd619, ALS_2ab02fd94858, ALS_a36b24bb1c30, ALS_7ce70862a85d, ALS_ef94089c7cf9, ALS_c16880baafef, ALS_642e1ed49c1d, ALS_dd460f732420, ALS_eb316782fac1, ALS_604d2b921cbd, ALS_550b790ed968, ALS_f404679703bb, ALS_96c0604adaed, ALS_7e3c7c19611e, ALS_49afc6509b03, ALS_199b4abda64a, ALS_e6a4d2c1e0a1, ALS_7872df4b70a2, ALS_1f8597314f50, ALS_73bc73616bed, ALS_db9ab9b91593, ALS_e4df1988b748, ALS_f4fcd56e8ce0, ALS_9da9c19fd142, ALS_ce782a02acc9, ALS_6ba8a27aef05, ALS_23cfd39a8ff5, ALS_c6c43ba02d91, ALS_6a83bd4f9193, ALS_e6de7cbe0e0f, ALS_6b406a241e64, ALS_318a370c3e19, ALS_3f1f4f2ebdd6, ALS_3528d6743c70, ALS_926c042075a4, ALS_ac3a58d2f36f, ALS_fc0d6ae93075, ALS_1291d0c97080, ALS_577004033855, ALS_1c29ac48e0ed, ALS_0933a1ad79af, ALS_b7764385162a, ALS_106c31937291, ALS_b863b66fc421, ALS_e8a5df7dbd55, ALS_6da65b132a69, ALS_c22e096d0305, ALS_b7f0ef257b87, ALS_e624559d5610, ALS_12fc406608bd, ALS_e81c9

In [67]:
transactions_shoes = spark.read.csv("Data_sets_articulos/transactions_shoes.csv", header = True)

In [68]:
transactions_shoes=transactions_shoes.withColumn("article_id",transactions_shoes.article_id.cast('integer'))
transactions_shoes=transactions_shoes.withColumn("client_id",transactions_shoes.client_id.cast('integer'))
transactions_shoes=transactions_shoes.withColumn("purchase_count",transactions_shoes.purchase_count.cast('integer'))

In [69]:
transactions_shoes

DataFrame[client_id: int, article_id: int, purchase_count: int]

In [70]:
(training, test) = transactions_shoes.randomSplit([0.8, 0.2])

In [142]:
# Lista donde guardar los resultados
ROEMS_SHOES = []

In [143]:
# Buscamos a través de los distintos modelos
for als in model_list:
    #Ajuste del modelo
    fitted_model = als.fit(training)
    predictions = fitted_model.transform(test)
    predictions = prediction_nan(predictions)
    ROEM_v = ROEM(predictions)

    # Guardamos los resultados y los vamos mostrando
    ROEMS_SHOES.append(ROEM_v)
    print ("Validation ROEM: ", ROEM_v)

Validation ROEM:  0.26958857944432285
Validation ROEM:  0.26540129352998715
Validation ROEM:  0.241778280863447
Validation ROEM:  0.2694199176402156
Validation ROEM:  0.26622082138106157
Validation ROEM:  0.2509270528224915
Validation ROEM:  0.2696833106679447
Validation ROEM:  0.2661400639448742
Validation ROEM:  0.2605261695692943
Validation ROEM:  0.26916292610824116
Validation ROEM:  0.2654191207038281
Validation ROEM:  0.20385700952004385
Validation ROEM:  0.2690419183347386
Validation ROEM:  0.26592085385850367
Validation ROEM:  0.23693766659533697
Validation ROEM:  0.2691163430066597
Validation ROEM:  0.266134156739345
Validation ROEM:  0.2611010840666273
Validation ROEM:  0.2687031933595653
Validation ROEM:  0.260285188423732
Validation ROEM:  0.24685803876587936
Validation ROEM:  0.26845471876723004
Validation ROEM:  0.26252361818687614
Validation ROEM:  0.24745051793544753
Validation ROEM:  0.26878714790329633
Validation ROEM:  0.262766101734188
Validation ROEM:  0.2515998793

**Modelo por tipo de producto. Underwear**

In [144]:
ranks = [10, 20, 30, 40]
maxIters = [5, 10]
regParams = [.001, .01, .1]
alphas = [1, 10, 100]

In [145]:
# For loop will automatically create and store ALS models
model_list = []
for r in ranks:
    for mi in maxIters:
        for rp in regParams:
            for a in alphas:
                model_list.append(ALS(userCol= "client_id", itemCol= "article_id", ratingCol= "purchase_count", rank = r, maxIter = mi, regParam = rp, alpha = a, coldStartStrategy="drop", nonnegative = True, implicitPrefs = True))

In [8]:
transactions_underwear = spark.read.csv("Data_sets_articulos/transactions_underwear.csv", header = True)

In [9]:
transactions_underwear=transactions_underwear.withColumn("article_id",transactions_underwear.article_id.cast('integer'))
transactions_underwear=transactions_underwear.withColumn("client_id",transactions_underwear.client_id.cast('integer'))
transactions_underwear=transactions_underwear.withColumn("purchase_count",transactions_underwear.purchase_count.cast('integer'))

In [10]:
transactions_underwear

DataFrame[client_id: int, article_id: int, purchase_count: int]

In [11]:
(training, test) = transactions_underwear.randomSplit([0.8, 0.2])

In [150]:
ROEMS_UNDERWEAR = []

In [151]:
# Buscamos a través de los distintos modelos
for als in model_list:
    #Ajuste del modelo
    fitted_model = als.fit(training)
    predictions = fitted_model.transform(test)
    predictions = prediction_nan(predictions)
    ROEM_v = ROEM(predictions)

    # Guardamos los resultados y los vamos mostrando
    ROEMS_UNDERWEAR.append(ROEM_v)
    print ("Validation ROEM: ", ROEM_v)

Validation ROEM:  0.34844122684139406
Validation ROEM:  0.34605885982167495
Validation ROEM:  0.0
Validation ROEM:  0.34822485151202404
Validation ROEM:  0.34602738289464324
Validation ROEM:  0.03446579422972201
Validation ROEM:  0.34834259940457185
Validation ROEM:  0.3459129524224723
Validation ROEM:  0.34602181662723597
Validation ROEM:  0.3480476322945817
Validation ROEM:  0.3449539897554647
Validation ROEM:  0.0
Validation ROEM:  0.3480503977976061
Validation ROEM:  0.34568724077898133
Validation ROEM:  0.3472861830832787
Validation ROEM:  0.3482622067742735
Validation ROEM:  0.3456286105106896
Validation ROEM:  0.3439032162150029
Validation ROEM:  0.34787168689561665
Validation ROEM:  0.3476328201198038
Validation ROEM:  0.2334356804577826
Validation ROEM:  0.3476057819397777
Validation ROEM:  0.34449748293323745
Validation ROEM:  0.2713967046255541
Validation ROEM:  0.3480439993227107
Validation ROEM:  0.3446826647730841
Validation ROEM:  0.33667136988453794
Validation ROEM:  0.

**Modelo por tipo de producto. Accessories**

In [6]:
ranks = [10, 20, 30, 40]
maxIters = [5, 10]
regParams = [.001, .01, .1]
alphas = [1, 10, 100]

In [7]:
# For loop will automatically create and store ALS models
model_list = []
for r in ranks:
    for mi in maxIters:
        for rp in regParams:
            for a in alphas:
                model_list.append(ALS(userCol= "client_id", itemCol= "article_id", ratingCol= "purchase_count", rank = r, maxIter = mi, regParam = rp, alpha = a, coldStartStrategy="drop", nonnegative = True, implicitPrefs = True))

In [69]:
transactions_accessories = spark.read.csv("Data_sets_articulos/transactions_accessories.csv", header = True)

In [70]:
transactions_accessories=transactions_accessories.withColumn("article_id",transactions_accessories.article_id.cast('integer'))
transactions_accessories=transactions_accessories.withColumn("client_id",transactions_accessories.client_id.cast('integer'))
transactions_accessories=transactions_accessories.withColumn("purchase_count",transactions_accessories.purchase_count.cast('integer'))

In [71]:
transactions_accessories

DataFrame[client_id: int, article_id: int, purchase_count: int]

In [72]:
(training, test) = transactions_accessories.randomSplit([0.8, 0.2])

In [16]:
ROEMS_ACCESSORIES = []

In [17]:
# Buscamos a través de los distintos modelos
for als in model_list:
    #Ajuste del modelo
    fitted_model = als.fit(training)
    predictions = fitted_model.transform(test)
    predictions = prediction_nan(predictions)
    ROEM_v = ROEM(predictions)

    # Guardamos los resultados y los vamos mostrando
    ROEMS_ACCESSORIES.append(ROEM_v)
    print ("Validation ROEM: ", ROEM_v)

Validation ROEM:  0.2987749817266608
Validation ROEM:  0.2982400581513609
Validation ROEM:  0.0
Validation ROEM:  0.29877528167169126
Validation ROEM:  0.2967508242336322
Validation ROEM:  0.29809036884216045
Validation ROEM:  0.29893438175177783
Validation ROEM:  0.29698507466947766
Validation ROEM:  0.2897848275217242
Validation ROEM:  0.29862991028530117
Validation ROEM:  0.29319739877808715
Validation ROEM:  0.0
Validation ROEM:  0.2986263336870434
Validation ROEM:  0.2968141873867864
Validation ROEM:  0.06881827289699793
Validation ROEM:  0.29878570887522693
Validation ROEM:  0.29692016054352754
Validation ROEM:  0.29123484205167566
Validation ROEM:  0.29813909356412127
Validation ROEM:  0.2927077818484795
Validation ROEM:  0.2623614830804094
Validation ROEM:  0.2979787761784997
Validation ROEM:  0.29397053486356345
Validation ROEM:  0.27201675504061706
Validation ROEM:  0.29870970962998633
Validation ROEM:  0.2943048123808547
Validation ROEM:  0.27967638624189534
Validation ROEM:

**Modelo por tipo de producto. Garment Lower body (GLB)**

In [18]:
ranks = [10, 20, 30, 40]
maxIters = [5, 10]
regParams = [.001, .01, .1]
alphas = [1, 10, 100]

In [19]:
# For loop will automatically create and store ALS models
model_list = []
for r in ranks:
    for mi in maxIters:
        for rp in regParams:
            for a in alphas:
                model_list.append(ALS(userCol= "client_id", itemCol= "article_id", ratingCol= "purchase_count", rank = r, maxIter = mi, regParam = rp, alpha = a, coldStartStrategy="drop", nonnegative = True, implicitPrefs = True))

In [103]:
transactions_glb = spark.read.csv("Data_sets_articulos/transactions_GLB.csv", header = True)

In [104]:
transactions_glb=transactions_glb.withColumn("article_id",transactions_glb.article_id.cast('integer'))
transactions_glb=transactions_glb.withColumn("client_id",transactions_glb.client_id.cast('integer'))
transactions_glb=transactions_glb.withColumn("purchase_count",transactions_glb.purchase_count.cast('integer'))

In [105]:
transactions_glb

DataFrame[client_id: int, article_id: int, purchase_count: int]

In [106]:
(training, test) = transactions_glb.randomSplit([0.8, 0.2])

In [25]:
ROEMS_GLB = []

In [26]:
# Buscamos a través de los distintos modelos
for als in model_list:
    #Ajuste del modelo
    fitted_model = als.fit(training)
    predictions = fitted_model.transform(test)
    predictions = prediction_nan(predictions)
    ROEM_v = ROEM(predictions)

    # Guardamos los resultados y los vamos mostrando
    ROEMS_GLB.append(ROEM_v)
    print ("Validation ROEM: ", ROEM_v)

Validation ROEM:  0.3977223422560737
Validation ROEM:  0.3963310924504021
Validation ROEM:  0.3991098848455961
Validation ROEM:  0.3977108216085837
Validation ROEM:  0.3964585843568265
Validation ROEM:  0.39687586696313043
Validation ROEM:  0.3972948455785168
Validation ROEM:  0.39644698751369095
Validation ROEM:  0.39750495223912835
Validation ROEM:  0.39755844843930555
Validation ROEM:  0.39597496731233894
Validation ROEM:  0.3978831267149171
Validation ROEM:  0.39749453457980416
Validation ROEM:  0.39593987028398736
Validation ROEM:  0.39602187392988714
Validation ROEM:  0.3974494450425451
Validation ROEM:  0.39591631677922984
Validation ROEM:  0.3968722308226483
Validation ROEM:  0.3974282725576753
Validation ROEM:  0.39698203376257263
Validation ROEM:  0.0
Validation ROEM:  0.3973329789944667
Validation ROEM:  0.3957313541950234
Validation ROEM:  0.06196671897401201
Validation ROEM:  0.3970908090244474
Validation ROEM:  0.39576039668699114
Validation ROEM:  0.3905734842595583
Vali

Las iteraciones anterirores muestran que exsiten combinaciones de parámetros en donde el ROEM es 0 o muy cercano a cero. La idea ahora es probar por medio de validación cruzada algunas de estas iteraciones hasta encontrar aquella que no tenga overfitting. 

#### Validación cruzada. Zapatos

En el caso del modelo para underwear, el ROEM mínimo obtenido fue de 0% con la siguiente combinación:

In [25]:
rank = 10
maxIter = 10
regParam = 0.001
alpha = 100

In [19]:
als = ALS(maxIter=maxIter, regParam=regParam, implicitPrefs=True, alpha = alpha, rank = rank, 
          userCol="client_id", itemCol="article_id", ratingCol="purchase_count",
          coldStartStrategy="nan", nonnegative = True)

In [14]:
# Generamos el split de la data de zapatos
(training, test) = transactions_shoes.randomSplit([0.8, 0.2])

#Construimos 5 folds
train1, train2, train3, train4, train5 = training.randomSplit([0.2, 0.2, 0.2, 0.2, 0.2], seed = 1)
fold1 = train2.union(train3).union(train4).union(train5)
fold2 = train3.union(train4).union(train5).union(train1)
fold3 = train4.union(train5).union(train1).union(train2)
fold4 = train5.union(train1).union(train2).union(train3)
fold5 = train1.union(train2).union(train3).union(train4)

foldlist = [(fold1, train1), (fold2, train2), (fold3, train3), (fold4, train4), (fold5, train5)]

In [15]:
for ft_pair in foldlist:
    modelo_adj = als.fit(ft_pair[0])
    predictions = modelo_adj.transform(ft_pair[1])
    predictions = prediction_nan(predictions)
    r = ROEM(predictions)
    print ("ROEM validacion: ", r)

ROEM validacion:  0.17711479961259533
ROEM validacion:  0.17398859524794894
ROEM validacion:  0.17498470078441417
ROEM validacion:  0.17450144303168066
ROEM validacion:  0.1805667136073882


In [16]:
# Ajustamos el model para todo el set de entrenamiento
v_fitted_model = als.fit(training)
v_predictions = v_fitted_model.transform(test)
v_predictions = prediction_nan(v_predictions)
v_ROEM = ROEM(v_predictions)

# Mostramos el ROEM para todo el set de validación
print ("ROEM final: ", v_ROEM)

ROEM final:  0.18107707215788987


#### Validación cruzada. Underwear

En el caso del modelo para underwear, el ROEM mínimo obtenido fue de 0.0% con la siguiente combinación:

In [34]:
rank = 20
maxIter = 10
regParam = 0.001
alpha = 100

In [35]:
als = ALS(maxIter=maxIter, regParam=regParam, implicitPrefs=True, alpha = alpha, rank = rank, 
          userCol="client_id", itemCol="article_id", ratingCol="purchase_count",
          coldStartStrategy="nan", nonnegative = True)

In [36]:
# Generamos el split de la data de zapatos
(training, test) = transactions_underwear.randomSplit([0.8, 0.2])

#Construimos 5 folds
train1, train2, train3, train4, train5 = training.randomSplit([0.2, 0.2, 0.2, 0.2, 0.2], seed = 1)
fold1 = train2.union(train3).union(train4).union(train5)
fold2 = train3.union(train4).union(train5).union(train1)
fold3 = train4.union(train5).union(train1).union(train2)
fold4 = train5.union(train1).union(train2).union(train3)
fold5 = train1.union(train2).union(train3).union(train4)

foldlist = [(fold1, train1), (fold2, train2), (fold3, train3), (fold4, train4), (fold5, train5)]

In [37]:
for ft_pair in foldlist:
    modelo_adj = als.fit(ft_pair[0])
    predictions = modelo_adj.transform(ft_pair[1])
    predictions = prediction_nan(predictions)
    r = ROEM(predictions)
    print ("ROEM validacion: ", r)

ROEM validacion:  0.2473329648214539
ROEM validacion:  0.24471072521113302
ROEM validacion:  0.2511109132209099
ROEM validacion:  0.2459025964806172
ROEM validacion:  0.24650881415255152


In [38]:
# Ajustamos el model para todo el set de entrenamiento
v_fitted_model = als.fit(training)
v_predictions = v_fitted_model.transform(test)
v_predictions = prediction_nan(v_predictions)
v_ROEM = ROEM(v_predictions)

# Mostramos el ROEM para todo el set de validación
print ("ROEM final: ", v_ROEM)

ROEM final:  0.17969235818227997


#### Validación cruzada. Accessories

En el caso del modelo para accesories, el ROEM mínimo obtenido fue de 0.0% con la siguiente combinación:

In [76]:
rank = 20
maxIter = 10
regParam = 0.001
alpha = 100

In [77]:
als = ALS(maxIter=maxIter, regParam=regParam, implicitPrefs=True, alpha = alpha, rank = rank, 
          userCol="client_id", itemCol="article_id", ratingCol="purchase_count",
          coldStartStrategy="nan", nonnegative = True)

In [78]:
# Generamos el split de la data de accesories
(training, test) = transactions_accessories.randomSplit([0.8, 0.2])

#Construimos 5 folds
train1, train2, train3, train4, train5 = training.randomSplit([0.2, 0.2, 0.2, 0.2, 0.2], seed = 1)
fold1 = train2.union(train3).union(train4).union(train5)
fold2 = train3.union(train4).union(train5).union(train1)
fold3 = train4.union(train5).union(train1).union(train2)
fold4 = train5.union(train1).union(train2).union(train3)
fold5 = train1.union(train2).union(train3).union(train4)

foldlist = [(fold1, train1), (fold2, train2), (fold3, train3), (fold4, train4), (fold5, train5)]

In [79]:
for ft_pair in foldlist:
    modelo_adj = als.fit(ft_pair[0])
    predictions = modelo_adj.transform(ft_pair[1])
    predictions = prediction_nan(predictions)
    r = ROEM(predictions)
    print ("ROEM validacion: ", r)

ROEM validacion:  0.1974840048075369
ROEM validacion:  0.20135811767107858
ROEM validacion:  0.1996095795397958
ROEM validacion:  0.20184915555507127
ROEM validacion:  0.20405009611624556


In [80]:
# Ajustamos el model para todo el set de entrenamiento
v_fitted_model = als.fit(training)
v_predictions = v_fitted_model.transform(test)
v_predictions = prediction_nan(v_predictions)
v_ROEM = ROEM(v_predictions)

# Mostramos el ROEM para todo el set de validación
print ("ROEM final: ", v_ROEM)

ROEM final:  0.22144255832142776


#### Validación cruzada. GLB

En el caso del modelo para GLB, el ROEM mínimo obtenido fue de 0.0% con la siguiente combinación:

In [107]:
rank = 30
maxIter = 10
regParam = 0.001
alpha = 100

In [108]:
als = ALS(maxIter=maxIter, regParam=regParam, implicitPrefs=True, alpha = alpha, rank = rank, 
          userCol="client_id", itemCol="article_id", ratingCol="purchase_count",
          coldStartStrategy="nan", nonnegative = True)

In [109]:
# Generamos el split de la data de accesories
(training, test) = transactions_glb.randomSplit([0.8, 0.2])

#Construimos 5 folds
train1, train2, train3, train4, train5 = training.randomSplit([0.2, 0.2, 0.2, 0.2, 0.2], seed = 1)
fold1 = train2.union(train3).union(train4).union(train5)
fold2 = train3.union(train4).union(train5).union(train1)
fold3 = train4.union(train5).union(train1).union(train2)
fold4 = train5.union(train1).union(train2).union(train3)
fold5 = train1.union(train2).union(train3).union(train4)

foldlist = [(fold1, train1), (fold2, train2), (fold3, train3), (fold4, train4), (fold5, train5)]

In [110]:
for ft_pair in foldlist:
    modelo_adj = als.fit(ft_pair[0])
    predictions = modelo_adj.transform(ft_pair[1])
    predictions = prediction_nan(predictions)
    r = ROEM(predictions)
    print ("ROEM validacion: ", r)

ROEM validacion:  0.2549466664197314
ROEM validacion:  0.26348012643012125
ROEM validacion:  0.25629603546534957
ROEM validacion:  0.2630299922191524
ROEM validacion:  0.26131367327428395


In [111]:
# Ajustamos el model para todo el set de entrenamiento
v_fitted_model = als.fit(training)
v_predictions = v_fitted_model.transform(test)
v_predictions = prediction_nan(v_predictions)
v_ROEM = ROEM(v_predictions)

# Mostramos el ROEM para todo el set de validación
print ("ROEM final: ", v_ROEM)

ROEM final:  0.20144370488121893


#### Analizando recomendaciones

#### Caso zapatos

In [65]:
rank = 10
maxIter = 10
regParam = 0.001
alpha = 100

In [66]:
als_shoes = ALS(maxIter=maxIter, regParam=regParam, implicitPrefs=True, alpha = alpha, rank = rank, 
          userCol="client_id", itemCol="article_id", ratingCol="purchase_count",
          coldStartStrategy="nan", nonnegative = True)

In [76]:
# Ajustamos el model para todo el set de entrenamiento
model_shoes = als_shoes.fit(training)
predictions = model_shoes.transform(test)
predictions = prediction_nan(predictions)
ROEM_v = ROEM(predictions)

# Mostramos el ROEM para todo el set de validación
print ("ROEM final: ", ROEM_v)

ROEM final:  0.1969880213627663


Generamos el top 5 de recomendaciones de zapatos para cada usuario

In [96]:
shoes_recommendation = model_shoes.recommendForAllUsers(5)

In [78]:
shoes_recommendation.show()

+---------+--------------------+
|client_id|     recommendations|
+---------+--------------------+
|      148|[{822355001, 0.33...|
|     1088|[{809521001, 1.15...|
|     1238|[{517762010, 1.03...|
|     1591|[{514134001, 0.77...|
|     1829|[{822355001, 0.50...|
|     1959|[{581275009, 0.12...|
|     2659|[{748588001, 0.97...|
|     3794|[{619234001, 0.38...|
|     3918|[{809521001, 0.78...|
|     4101|[{258650001, 0.0}...|
|     4818|[{514134001, 0.77...|
|     4935|[{212042070, 0.0}...|
|     5156|[{619235001, 0.76...|
|     5300|[{581275009, 1.09...|
|     5803|[{619235001, 0.37...|
|     6336|[{622956001, 0.77...|
|     6357|[{514134001, 0.73...|
|     6397|[{762977001, 0.78...|
|     6620|[{514134001, 0.77...|
|     7253|[{748588001, 1.42...|
+---------+--------------------+
only showing top 20 rows



In [100]:
shoes_recommendation = shoes_recommendation.withColumn("recomm", explode("recommendations")).select('client_id', col("recomm.article_id"), col("recomm.rating"))

In [101]:
shoes_recommendationDF = shoes_recommendation.toPandas()

In [103]:
shoes_recommendationDF.head(10)

,client_id,article_id,rating
0,148,822355001,0.338861
1,148,593808005,0.337656
2,148,694614001,0.331626
3,148,808685001,0.324381
4,148,606711001,0.323425
5,1088,809521001,1.158062
6,1088,808666003,1.144829
7,1088,655287002,1.129071
8,1088,853931004,1.120902
9,1088,758472002,1.093903


In [105]:
shoes_recommendationDF.to_csv("shoes_recommendation.csv")

#### Caso underwear

In [21]:
rank = 20
maxIter = 10
regParam = 0.001
alpha = 100

In [22]:
als_underwear = ALS(maxIter=maxIter, regParam=regParam, implicitPrefs=True, alpha = alpha, rank = rank, 
          userCol="client_id", itemCol="article_id", ratingCol="purchase_count",
          coldStartStrategy="nan", nonnegative = True)

In [25]:
# Ajustamos el model para todo el set de entrenamiento
model_underwear = als_underwear.fit(training)
predictions = model_underwear.transform(test)
predictions = prediction_nan(predictions)
ROEM_v = ROEM(predictions)

# Mostramos el ROEM para todo el set de validación
print ("ROEM final: ", ROEM_v)

ROEM final:  0.1740319013924613


In [26]:
underwear_recommendation = model_underwear.recommendForAllUsers(5)

In [27]:
underwear_recommendation = underwear_recommendation.withColumn("recomm", explode("recommendations")).select('client_id', col("recomm.article_id"), col("recomm.rating"))

In [28]:
underwear_recommendationDF = underwear_recommendation.toPandas()

In [29]:
underwear_recommendationDF.head(10)

,client_id,article_id,rating
0,148,688728018,0.721091
1,148,598515020,0.707650
2,148,614854016,0.677203
3,148,564313003,0.670933
4,148,854322001,0.652681
5,463,686259004,0.748212
6,463,735607005,0.738398
7,463,761113002,0.704035
8,463,688728018,0.702887
9,463,678687016,0.699006


In [39]:
underwear_recommendationDF.to_csv("underwear_recommendation.csv")

#### Caso Accessories

In [61]:
rank = 20
maxIter = 10
regParam = 0.001
alpha = 100

In [62]:
als_accessories = ALS(maxIter=maxIter, regParam=regParam, implicitPrefs=True, alpha = alpha, rank = rank, 
          userCol="client_id", itemCol="article_id", ratingCol="purchase_count",
          coldStartStrategy="nan", nonnegative = True)

In [63]:
# Ajustamos el model para todo el set de entrenamiento
model_accessories = als_accessories.fit(training)
predictions = model_accessories.transform(test)
predictions = prediction_nan(predictions)
ROEM_v = ROEM(predictions)

# Mostramos el ROEM para todo el set de validación
print ("ROEM final: ", ROEM_v)

ROEM final:  0.22246699241399884


In [65]:
accessories_recommendation = model_accessories.recommendForAllUsers(5)

In [66]:
accessories_recommendation = accessories_recommendation.withColumn("recomm", explode("recommendations")).select('client_id', col("recomm.article_id"), col("recomm.rating"))

In [67]:
accessories_recommendationDF = accessories_recommendation.toPandas()

In [75]:
accessories_recommendationDF.head(10)

,client_id,article_id,rating
0,471,523488001,0.949682
1,471,560047001,0.949671
2,471,749974001,0.822573
3,471,318066004,0.811736
4,471,732409003,0.779234
5,833,453358001,0.763221
6,833,523490002,0.760363
7,833,647982001,0.739034
8,833,337991001,0.701750
9,833,648904001,0.696048


In [73]:
accessories_recommendationDF.to_csv("accessories_recommendation.csv")

#### Caso Garment Lower body (GLB)

In [94]:
rank = 30
maxIter = 10
regParam = 0.001
alpha = 100

In [95]:
als_glb = ALS(maxIter=maxIter, regParam=regParam, implicitPrefs=True, alpha = alpha, rank = rank, 
          userCol="client_id", itemCol="article_id", ratingCol="purchase_count",
          coldStartStrategy="nan", nonnegative = True)

In [96]:
# Ajustamos el model para todo el set de entrenamiento
model_glb = als_glb.fit(training)
predictions = model_glb.transform(test)
predictions = prediction_nan(predictions)
ROEM_v = ROEM(predictions)

# Mostramos el ROEM para todo el set de validación
print ("ROEM final: ", ROEM_v)

ROEM final:  0.19875352309897756


In [97]:
glb_recommendation = model_glb.recommendForAllUsers(5)

In [98]:
glb_recommendation = glb_recommendation.withColumn("recomm", explode("recommendations")).select('client_id', col("recomm.article_id"), col("recomm.rating"))

In [99]:
glb_recommendationDF = glb_recommendation.toPandas()

In [100]:
glb_recommendationDF.head(10)

,client_id,article_id,rating
0,148,646429003,0.711674
1,148,597891012,0.629168
2,148,636902001,0.611342
3,148,749509006,0.603256
4,148,754176001,0.601963
5,463,680327001,0.787323
6,463,573716046,0.784925
7,463,719463002,0.750850
8,463,751471039,0.735637
9,463,690935001,0.706478


In [102]:
glb_recommendationDF.to_csv("germent_lower_body_recommendation.csv")

In [90]:
predictions.show()

+---------+----------+--------------+-------------------+
|client_id|article_id|purchase_count|         prediction|
+---------+----------+--------------+-------------------+
|   959921| 241412052|             1|                0.0|
|  1171601| 241412052|             1|                0.0|
|   180359| 241412052|             1|                0.0|
|    16119| 241412052|             2|                0.0|
|   183701| 241412052|             1|                0.0|
|   208332| 241412052|             1|                0.0|
|   129384| 241412052|             1|                0.0|
|  1036426| 241412052|             1|                0.0|
|    20800| 241412052|             1|                0.0|
|    31581| 241412052|             1|                0.0|
|   761755| 241412052|             1|                0.0|
|  1071151| 241412052|             1|0.15633703486100303|
|   878168| 241412052|             1|                0.0|
|  1061371| 241412052|             2|                0.0|
|   967248| 24

In [93]:
glb_recommendation.show()

+---------+----------+------+
|client_id|article_id|rating|
+---------+----------+------+
|      148| 156610010|   0.0|
|      148| 179123040|   0.0|
|      148| 194242050|   0.0|
|      148| 249136020|   0.0|
|      148| 250457010|   0.0|
|      463| 156610010|   0.0|
|      463| 179123040|   0.0|
|      463| 194242050|   0.0|
|      463| 249136020|   0.0|
|      463| 250457010|   0.0|
|      471| 156610010|   0.0|
|      471| 179123040|   0.0|
|      471| 194242050|   0.0|
|      471| 249136020|   0.0|
|      471| 250457010|   0.0|
|      833| 156610010|   0.0|
|      833| 179123040|   0.0|
|      833| 194242050|   0.0|
|      833| 249136020|   0.0|
|      833| 250457010|   0.0|
+---------+----------+------+
only showing top 20 rows

